<a href="https://colab.research.google.com/github/paolotous/ContactManager/blob/WholeProject/practice_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
"""Starter code"""
 
SPECIAL_CASE_SCHOOL_1 = 'Fort McMurray Composite High'
SPECIAL_CASE_SCHOOL_2 = 'Father Mercredi High School'
SPECIAL_CASE_YEAR = '2017'
 
# Add other constants here
 
 
def is_special_case(record: str) -> bool:
    """Return True iff the student represented by record is a special case.
 
    >>> is_special_case('Jacqueline Smith,Fort McMurray Composite High,2017,MAT,90,94,ENG,92,88,CHM,80,85,BArts')
    True
    >>> is_special_case('Jacqueline Smith,Father Something High School,2017,MAT,90,94,ENG,92,88,CHM,80,85,BArts')
    False
    >>> is_special_case('Jacqueline Smith,Fort McMurray Composite High,2015,MAT,90,94,ENG,92,88,CHM,80,85,BArts')
    False
    """
    separate = record.split(",")
    if (separate[1] == SPECIAL_CASE_SCHOOL_1 or separate[1] == SPECIAL_CASE_SCHOOL_2) and separate[2] == SPECIAL_CASE_YEAR:
      return True
    else:
      return False
    # Complete the body of the function here
 
 
# Complete the rest of the functions here
def get_final_mark(record: str, coursework: str, examwork: str) -> float:
  if examwork == "NE":
    if is_special_case(record):
      final = coursework
      return final
    else:
      examwork = 0
  final = (float(coursework) + float(examwork)) / 2      
  return float(final)    

def get_both_marks(record: str, courseCode: str) -> str:
  separate = record.split(",")
  for i in range(len(separate)):
    if separate[i] == courseCode:
      return str(separate[i+1]) + " " + str(separate[i+2])
  return ""

def extract_course(record: str, courseIndex: int) -> str:
  counter = 0
  separate = record.split(",")
  if courseIndex >= 1:
    for i in range(len(separate)):
      if separate[i].isalpha() and len(separate[i]) == 3:
        counter += 1
        if counter == courseIndex:
          return separate[i] + "," + str(separate[i+1]) + "," + str(separate[i+2])
  return ""

def applied_to_degree(record: str, degree: str) -> bool:
  separate = record.split(",")
  for i in range(len(separate)):
    if separate[i] == degree:
      return True
  return False

#Next -> split the cutoff to take the average required and compare it with the average in the first parameter, watch out for spacing
#The following can replace multiple special characters by one shared character and then split (instead of using regex)
#'a;bcd,ef g'.replace(';',' ').replace(',',' ').split()
#['a', 'bcd', 'ef', 'g']
def decide_admission(studentAverage: int, cutoff: int) -> str:
  if studentAverage - cutoff >= 5:
    return "accept with scholarship"
  if studentAverage >= cutoff:
    return "accept"
  return "reject"    


#Testing Area
print("So far so good")
is_special_case('Jacqueline Smith,Fort McMurray Composite High,2017,MAT,90,94,ENG,92,88,CHM,80,85,BArts')
#is_special_case('Jacqueline Smith,Father Something High School,2017,MAT,90,94,ENG,92,88,CHM,80,85,BArts')
#is_special_case('Jacqueline Smith,Fort McMurray Composite High,2015,MAT,90,94,ENG,92,88,CHM,80,85,BArts')
final = get_final_mark('Jacqueline Smith,Fort McMurray Composite High,2017', 90, 'NE')
#get_final_mark('Jacqueline Smith,Fort McMurray Composite High,2017', 90, 80)
#get_final_mark('Jacqueline Smith,Fort McMurray Composite High,2018', 90, 0)
both_mark = get_both_marks("MAT,90,94", "MAT")
extract_course("MAT,90,94,ENG,92,NE,CHM,80,85", 3)
applied_to_degree('Jacqueline Smith,Fort McMurray Composite High,2017,MAT,90,94,ENG,92,88,CHM,80,85,BArts', 'BArts')
decision = decide_admission(83, 80)

So far so good


In [10]:
"""Using functions from admission_functions.py"""

#import admission_functions as af  -- no need to since I'm using Colab
from typing import List

# Format of a student record
# name,school,gradyear,course1,grade1,exam1,course2,grade2,exam2,course3,grade3,exam3,degree

# NOTE: For A1, lines containing student records can exceed 80 characters.
# All other lines must stay under the 80 character limit.

NAME_INDEX = 0
TRANSCRIPT_INDEX = 3
NUM_APPLICATION_COURSES = 3
MARK_LENGTH = 2


def process_student_records(records: List[str]) -> None: 
    """ Main function for processing application 
    records in the admission system.
    """

    for record in records:
        record_data = record.split(',')
        transcript = ','.join(record_data[TRANSCRIPT_INDEX:-1])

        mark_list = get_marks(record, transcript)
        determine_degree_admission(record, record_data[NAME_INDEX], mark_list)


def get_marks(record: str, transcript: str) -> List[float]:
    """  Return a list of final course marks from transcript that can be used for
    admission, computed based on information in record.

    >>> record = 'Karen Reid,Fort McMurray Composite High,2017,MAT,90,92,ENG,92,NE,BIO,77,85,BSci'
    >>> transcript = 'MAT,90,92,ENG,92,NE,BIO,77,85'
    >>> get_marks(record, transcript)
    [91.0, 92.0, 81.0]
    >>> record = 'Paul Gries,Ithaca High School,1986,BIO,61,70,CHM,80,90,CAT,95,96,BEng'
    >>> transcript = 'BIO,61,70,CHM,80,90,CAT,95,96'
    >>> get_marks(record, transcript)
    [65.5, 85.0]
    """
    
    valid_admission_courses = ['BIO', 'CHM', 'ENG', 'MAT', 'SCI']
    mark_list = []

    for i in range(1, NUM_APPLICATION_COURSES + 1):
        course = extract_course(transcript, i)
        for course_code in valid_admission_courses:
            both_marks = get_both_marks(course, course_code)
            if len(both_marks) > 0:
                course_mark = both_marks[:MARK_LENGTH]
                exam_mark = both_marks[MARK_LENGTH + 1:]
                final_mark = get_final_mark(record, course_mark, exam_mark)
                mark_list.append(float(final_mark))
    return mark_list


def determine_degree_admission(record: str, name: str, 
                               mark_list: List[float]) -> None:
    """Prints the admission decision for each record containing name and the
    decision based on the average of mark_list.

    >>> record = 'Jacqueline Smith,Grande Prairie Composite High,2017,MAT,90,94,ENG,92,88,CHM,80,85,BArts'
    >>> name = 'Jacqueline Smith'
    >>> determine_degree_admission(record, name, [92.0, 90.0, 82.5])
    BArts admission decision for Jacqueline Smith: accept with scholarship
    >>> record = 'Paul Gries,Ithaca High School,1986,BIO,60,70,CHM,80,90,CAT,95,96,BEng'
    >>> determine_degree_admission(record, 'Paul Gries', [65.0, 85.0])
    BEng admission decision for Paul Gries: reject
    """

    degrees = {'BArts': 80, 'BCom': 82, 'BEng': 84, 'BMusic': 86, 'BSci': 88}

    for degree in degrees:
        if applied_to_degree(record, degree):
            average = sum(mark_list) / NUM_APPLICATION_COURSES
            cutoff = degrees[degree]
            decision = decide_admission(average, cutoff)
            print('{0} admission decision for {1}: {2}'.format(degree, name,
                                                               decision))


if __name__ == '__main__':
    import doctest
    doctest.testmod()

    records_string = \
        '''Jacqueline Smith,Best High School,2002,MAT,90,94,ENG,92,88,CHM,80,85,BArts
Karen Reid,Fort McMurray Composite High,2017,MAT,90,92,ENG,92,NE,BIO,77,85,BSci
Paul Gries,Ithaca High School,1986,BIO,60,70,CHM,80,90,CAT,95,96,BEng
Andrew Petersen,Some Other High School,2017,BIO,83,86,MAT,91,84,SCI,95,77,BMusic
Jen Campbell,Yet Another High School,2015,ENG,75,78,SCI,80,81,CHM,80,81,BCom'''

    records = records_string.split('\n')
    process_student_records(records)


BArts admission decision for Jacqueline Smith: accept with scholarship
BSci admission decision for Karen Reid: accept
BEng admission decision for Paul Gries: reject
BMusic admission decision for Andrew Petersen: accept
BCom admission decision for Jen Campbell: reject
